In [1]:
import matplotlib.pyplot as plt
import itertools
import math
import pandas as pd
import os
import numpy as np

In [2]:
ner_df = pd.read_csv('100 sentences.csv')

In [3]:
ner_df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,label
0,1400,Patient with complaint of a very painful left...,SOAP / Chart / Progress Notes,Foot Lesions,"S -, An 84-year-old diabetic female, 5'7-1/2"" ...","soap / chart / progress notes, painful left fo...",O O O O O O B-Symptoms I-Symptoms I-Symptoms O...
1,1401,Evaluation and recommendations regarding faci...,SOAP / Chart / Progress Notes,Facial Rhytids,"HISTORY: , This 57-year-old female who present...","soap / chart / progress notes, quick lift, hyp...",O O O O B-Symptoms I-Symptoms O
2,1402,Followup 4 months status post percutaneous sc...,SOAP / Chart / Progress Notes,Followup Screw Fixation,"REASON FOR VISIT: ,Followup 4 months status p...","soap / chart / progress notes, metatarsal head...",O O O O O B-Treatment I-Treatment I-Treatment ...
3,1403,"Dietary consultation for hyperlipidemia, hype...",SOAP / Chart / Progress Notes,Dietary Consult - Weight Reduction,"SUBJECTIVE:, This is a 56-year-old female who...","soap / chart / progress notes, hyperlipidemia,...",B-Treatment I-Treatment O B-Symptoms O B-Sympt...
4,1404,Recurrent urinary tract infection in a patien...,SOAP / Chart / Progress Notes,E. Coli UTI - Followup,"HISTORY OF PRESENT ILLNESS:, The patient pres...","soap / chart / progress notes, urinary tract i...",O B-Symptoms I-Symptoms I-Symptoms O O O O O O...


In [6]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package punkt to /home/tities/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
# Function to get unique words to populate vocab

def getUniqueWords(listOfWords):
  uniqueWords = [] 
  for words in listOfWords:
    for word in words:
      if not word in uniqueWords:
        uniqueWords.append(word)

  return uniqueWords

In [11]:
# Tokenize sentences into words, get unique words, divide dataset into data training and data test

sentences_word = []
sentences_label = []
MAX_LEN=0
# tokenize sentence and label
for sentences, labels in ner_df[['description','label']].values:
  sentences_word.append(word_tokenize(sentences))
  sentences_label.append(word_tokenize(labels))
  # get maximum length of words in a sentence
  if MAX_LEN < len(word_tokenize(sentences)):
    MAX_LEN = len(word_tokenize(sentences))

# get lists of labels available and populate vocab
unique_labels = getUniqueWords(sentences_label)
unique_words = getUniqueWords(sentences_word)

# get vocab size
VOCAB_SIZE = len(unique_words)
sentences_label

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-Symptoms',
  'I-Symptoms',
  'I-Symptoms',
  'O',
  'O',
  'O',
  'B-Symptoms',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-Anatomy',
  'O'],
 ['O', 'O', 'O', 'O', 'B-Symptoms', 'I-Symptoms', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'B-Treatment',
  'I-Treatment',
  'I-Treatment',
  'O',
  'O',
  'O',
  'B-Anatomy',
  'O',
  'B-Symptoms',
  'I-Symptoms',
  'I-Symptoms',
  'O',
  'O',
  'O',
  'O',
  'B-Symptoms',
  'I-Symptoms',
  'I-Symptoms',
  'O',
  'O',
  'O'],
 ['B-Treatment',
  'I-Treatment',
  'O',
  'B-Symptoms',
  'O',
  'B-Symptoms',
  'O',
  'B-Symptoms',
  'I-Symptoms',
  'I-Symptoms',
  'O',
  'B-Treatment',
  'I-Treatment',
  'O'],
 ['O',
  'B-Symptoms',
  'I-Symptoms',
  'I-Symptoms',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-Symptoms',
  'I-Symptoms',
  'I-Symptoms',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',


In [12]:
from sklearn.utils import shuffle

In [13]:
#shuffle dataset
sentences_word = np.array(sentences_word,dtype=object)
sentences_label = np.array(sentences_label,dtype=object)
sentences_word, sentences_label = shuffle(sentences_word, sentences_label)

# 60% of dataset assigned to be data training
train_size = int(len(sentences_word) * 0.6)

# divide dataset to data training and data set
train_sentences_words = sentences_word[:train_size]
train_sentences_labels = sentences_label[:train_size]
test_sentences_words = sentences_word[train_size:]
test_sentences_labels = sentences_label[train_size:]

In [14]:
#indexing vocab
words_tokenizer = Tokenizer(num_words=VOCAB_SIZE+2, filters=[], oov_token='__UNKNOWN__')
words_tokenizer.fit_on_texts(map(lambda s: ' '.join(s), sentences_word))
word_index = words_tokenizer.word_index
word_index['__PADDING__'] = 0
index_word = {i:w for w, i in word_index.items()}
print(index_word)

{1: '__UNKNOWN__', 2: '.', 3: ',', 4: 'with', 5: 'a', 6: 'of', 7: 'and', 8: 'the', 9: 'for', 10: 'patient', 11: 'to', 12: 'female', 13: 'in', 14: 'followup', 15: 'is', 16: 'sleep', 17: 'was', 18: 'on', 19: 'diabetes', 20: 'chronic', 21: 'an', 22: 'ultrasound', 23: 'history', 24: 'right', 25: 'pain', 26: 'his', 27: '-', 28: 'her', 29: 'dietary', 30: 'consultation', 31: 'disease', 32: 'noted', 33: 'white', 34: 'eeg', 35: 'care', 36: 'has', 37: 'type', 38: 'she', 39: 'presents', 40: 'arthritis', 41: 'no', 42: 'activity', 43: 'pregnancy', 44: 'at', 45: 'from', 46: 'status', 47: 'due', 48: 'syndrome', 49: 'this', 50: 'lower', 51: 'apnea', 52: 'mellitus', 53: 'who', 54: 'had', 55: 'left', 56: '1', 57: 'abnormal', 58: 'symptoms', 59: 'male', 60: 'during', 61: 'weight', 62: 'today', 63: 'diabetic', 64: 'control', 65: 'well', 66: 'evaluation', 67: 'medical', 68: 'note', 69: 'post', 70: 'oximetry', 71: 'visit', 72: 'atrial', 73: 'hyperlipidemia', 74: 'foot', 75: 'electroencephalogram', 76: 'preg

In [15]:
#convert words to sequence based on above index
train_sequences = words_tokenizer.texts_to_sequences(map(lambda s: ' '.join(s), train_sentences_words))
test_sequences = words_tokenizer.texts_to_sequences(map(lambda s: ' '.join(s), test_sentences_words))

In [16]:
#padding array to get the same size for all arrays
train_sequences_padded = pad_sequences(train_sequences, maxlen=MAX_LEN)
test_sequences_padded = pad_sequences(test_sequences, maxlen=MAX_LEN)

In [17]:
#repeat process for label
labels_tokenizer = Tokenizer(num_words=len(unique_labels)+2, filters='', oov_token='__UNKNOWN__', lower=False)
labels_tokenizer.fit_on_texts(map(lambda s: ' '.join(s), sentences_label))
label_index = labels_tokenizer.word_index
label_index['__PADDING__'] = 0
index_label = {i:w for w, i in label_index.items()}
print(index_label)

index_label_wo_padding = dict(index_label)
index_label_wo_padding[label_index['__PADDING__']] = '0'

{1: '__UNKNOWN__', 2: 'O', 3: 'B-Symptoms', 4: 'I-Symptoms', 5: 'B-Treatment', 6: 'I-Treatment', 7: 'B-Anatomy', 8: 'I-Anatomy', 9: 'I-Treatmet', 0: '__PADDING__'}


In [18]:
train_labels = labels_tokenizer.texts_to_sequences(map(lambda s: ' '.join(s), train_sentences_labels))
test_labels = labels_tokenizer.texts_to_sequences(map(lambda s: ' '.join(s), test_sentences_labels))

In [19]:
train_labels_padded = pad_sequences(train_labels, maxlen=MAX_LEN)
test_labels_padded = pad_sequences(test_labels, maxlen=MAX_LEN)

In [20]:
#check data mapping
for w, t in zip(test_sequences[16], test_labels[16]):
    print (index_word[w], index_label[t])

a O
sample O
note O
on O
rheumatoid B-Symptoms
arthritis I-Symptoms


In [21]:
# BiLSTM model

In [22]:
from keras.layers import Dense, Input, LSTM, Embedding, Bidirectional, Dropout
from keras.models import Model
from keras.initializers import Constant

In [23]:
random_embedding_layer = Embedding(VOCAB_SIZE,
                                300,
                                input_length=MAX_LEN)

sequence_input = Input(shape=(MAX_LEN,), dtype='int32')
embedded_sequences = random_embedding_layer(sequence_input)
x = Bidirectional(LSTM(64, return_sequences=True))(embedded_sequences)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
preds = Dense(len(label_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['sparse_categorical_accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 41)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 41, 300)           195300    
_________________________________________________________________
bidirectional (Bidirectional (None, 41, 128)           186880    
_________________________________________________________________
dropout (Dropout)            (None, 41, 128)           0         
_________________________________________________________________
dense (Dense)                (None, 41, 32)            4128      
_________________________________________________________________
dense_1 (Dense)              (None, 41, 10)            330       
Total params: 386,638
Trainable params: 386,638
Non-trainable params: 0
_______________________________________________________

In [24]:
model.fit(train_sequences_padded, train_labels_padded,
          batch_size=32,
          epochs=100,
          validation_data=(test_sequences_padded, test_labels_padded))

Epoch 1/100
2/2 [==============================] - 3s 774ms/step - loss: 2.2927 - sparse_categorical_accuracy: 0.2049 - val_loss: 2.2138 - val_sparse_categorical_accuracy: 0.6707
Epoch 2/100
2/2 [==============================] - 0s 88ms/step - loss: 2.1833 - sparse_categorical_accuracy: 0.6203 - val_loss: 2.0538 - val_sparse_categorical_accuracy: 0.6683
Epoch 3/100
2/2 [==============================] - 0s 90ms/step - loss: 2.0205 - sparse_categorical_accuracy: 0.6171 - val_loss: 1.8135 - val_sparse_categorical_accuracy: 0.6677
Epoch 4/100
2/2 [==============================] - 0s 101ms/step - loss: 1.7851 - sparse_categorical_accuracy: 0.6175 - val_loss: 1.4928 - val_sparse_categorical_accuracy: 0.6677
Epoch 5/100
2/2 [==============================] - 0s 96ms/step - loss: 1.4876 - sparse_categorical_accuracy: 0.6171 - val_loss: 1.1579 - val_sparse_categorical_accuracy: 0.6677
Epoch 6/100
2/2 [==============================] - 0s 98ms/step - loss: 1.2008 - sparse_categorical_accuracy

Epoch 47/100
2/2 [==============================] - 0s 84ms/step - loss: 0.2549 - sparse_categorical_accuracy: 0.8996 - val_loss: 0.2454 - val_sparse_categorical_accuracy: 0.9238
Epoch 48/100
2/2 [==============================] - 0s 114ms/step - loss: 0.2455 - sparse_categorical_accuracy: 0.9020 - val_loss: 0.2454 - val_sparse_categorical_accuracy: 0.9238
Epoch 49/100
2/2 [==============================] - 0s 89ms/step - loss: 0.2329 - sparse_categorical_accuracy: 0.9081 - val_loss: 0.2447 - val_sparse_categorical_accuracy: 0.9238
Epoch 50/100
2/2 [==============================] - 0s 82ms/step - loss: 0.2240 - sparse_categorical_accuracy: 0.9126 - val_loss: 0.2447 - val_sparse_categorical_accuracy: 0.9244
Epoch 51/100
2/2 [==============================] - 0s 90ms/step - loss: 0.2178 - sparse_categorical_accuracy: 0.9130 - val_loss: 0.2468 - val_sparse_categorical_accuracy: 0.9250
Epoch 52/100
2/2 [==============================] - 0s 92ms/step - loss: 0.2073 - sparse_categorical_acc

Epoch 93/100
2/2 [==============================] - 0s 130ms/step - loss: 0.0259 - sparse_categorical_accuracy: 0.9943 - val_loss: 0.2811 - val_sparse_categorical_accuracy: 0.9390
Epoch 94/100
2/2 [==============================] - 0s 92ms/step - loss: 0.0267 - sparse_categorical_accuracy: 0.9935 - val_loss: 0.3098 - val_sparse_categorical_accuracy: 0.9317
Epoch 95/100
2/2 [==============================] - 0s 62ms/step - loss: 0.0244 - sparse_categorical_accuracy: 0.9943 - val_loss: 0.2894 - val_sparse_categorical_accuracy: 0.9372
Epoch 96/100
2/2 [==============================] - 0s 60ms/step - loss: 0.0228 - sparse_categorical_accuracy: 0.9947 - val_loss: 0.2781 - val_sparse_categorical_accuracy: 0.9390
Epoch 97/100
2/2 [==============================] - 0s 79ms/step - loss: 0.0212 - sparse_categorical_accuracy: 0.9976 - val_loss: 0.2895 - val_sparse_categorical_accuracy: 0.9384
Epoch 98/100
2/2 [==============================] - 0s 129ms/step - loss: 0.0205 - sparse_categorical_ac

In [25]:
# Evaluation

In [26]:
from seqeval.metrics import classification_report, f1_score

In [27]:
lstm_predicted = model.predict(test_sequences_padded)

In [28]:
lstm_predicted_labels = []

for s, s_pred in zip(test_sentences_words, lstm_predicted):
    labels = np.argmax(s_pred, axis=1)
    labels = list(map(index_label_wo_padding.get,labels))[-len(s):]
    lstm_predicted_labels.append(labels)
print(lstm_predicted_labels)

[['B-Symptoms', 'O', 'I-Symptoms', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Symptoms', 'I-Symptoms', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['0', 'O', 'O', 'B-Symptoms', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-Treatment', 'O', 'B-Treatment', 'O', 'O', 'O', 'O', 'B-Treatment', 'O', 'B-Treatment', 'I-Treatment', 'O', 'B-Treatment', 'I-Treatment', 'O', 'B-Treatment', 'O', 'O', 'O', 'O'], ['B-Symptoms', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'I-Treatment', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'I-Symptoms', 'I-Symptoms', 'I-Symptoms', 'O', 'O', 'B-Symptoms', 'I-Symptoms', 'O'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'O', 'O', 'O', '0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Symptoms', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'I-Tr

In [29]:
print ('LSTM')
print ('='*15)
print (classification_report(test_sentences_labels, lstm_predicted_labels))

LSTM
              precision    recall  f1-score   support

     Anatomy       0.00      0.00      0.00         3
    Symptoms       0.35      0.28      0.31        40
   Treatment       0.62      0.43      0.51        23
    Treatmet       0.00      0.00      0.00         1

   micro avg       0.45      0.31      0.37        67
   macro avg       0.24      0.18      0.21        67
weighted avg       0.43      0.31      0.36        67



/home/tities/anaconda3/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/tities/anaconda3/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
